# Vocal apparatus V. Wetzel Simulations
This notebook is used to run simulations od the vocal tract and store results for later analysis.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import sympy as sp

import os, sys

p = os.path.abspath('..')
sys.path.insert(1, p)

import rpm_solver as solver
import models
import vocal_tract.Articulation as art
from scipy.io.wavfile import write

# Dirac excitation

## Multiple tracts (non shifted), case N_tract= 10, change of sampling frequency

In [2]:
# Geometry parameters and model initialization
N = 10
l0 = 0.17/(2*N)
L0 = 0.01
Sw = 2*l0*L0
k_s = 4e6
k = 845 #k_s*Sw
r = 0.8
h0 = 1e-2

model = models.MultipleTractsWithWalls(N, l0, Sw, k, r)

In [3]:
# Initialization and input signals
rhoInit = model.rho0
hInit = L0
mInit = rhoInit*model.Sw*hInit

init = np.zeros(model.n_state)

for i in range(N):
    init[i*6+3] = Sw * model.rho0 * h0 
    init[i*6+4] = h0 


In [17]:
# Simulations loop
duration = 0.1

srs = [2000, 5000, 10000, 20000]

for sr in srs:
    timestep = 1/sr
    rpmsolver = solver.RPMSolverPHS(model, 1, 1, timestep, max_iter=200,  epsilon = 10**(-10), quad_order = 100)

    # Dirac excitation
    def input_signal(t):
        sig = np.zeros((len(t), model.n_io))
        sig[:, 0] =   2e-4 * (t<1/sr)
        return sig
    proj_input_signal = solver.adc_dac.discretize_Bo0(rpmsolver, input_signal, duration)

    xframes, dx_proj, w_proj, y_proj, lambda_proj, iters = rpmsolver.simulate(init, duration, proj_input_signal, f'results/dirac/10_tracts/{sr}_Hz.pkl')
    print(f'Simulation for sr = {sr} Hz finished')

Mean number of NR iterations : 3.005
Max number of NR iterations : 4.0,              step index : 1
Simulation for sr = 2000 Hz finished
Mean number of NR iterations : 3.0
Max number of NR iterations : 3.0,              step index : 0
Simulation for sr = 5000 Hz finished
Mean number of NR iterations : 3.0
Max number of NR iterations : 3.0,              step index : 0
Simulation for sr = 10000 Hz finished
Mean number of NR iterations : 3.0
Max number of NR iterations : 3.0,              step index : 0
Simulation for sr = 20000 Hz finished


## Multiple tracts (non shifted), case sr = 20 kHz, change of N_tracts

In [51]:
# (Some) geometry parameters and input signal
duration = 0.2
sr = 40000
timestep = 1/sr

# Dirac excitation
def input_signal(t):
    sig = np.zeros((len(t), model.n_io))
    sig[:, 0] =   2e-4*(t>100/sr)
    return sig

L0 = 0.01
k_s = 4e6
k = k_s*Sw # 845
r = 0.8
h0 = 0.01

In [53]:
# Simulations loop
N_tracts = [2]#, 4, 6, 8, 10, 12, 14, 16]

for N in N_tracts:
    l0 = 0.17/(2*N)
    Sw = 2*l0*L0
    # Model generation
    model = models.MultipleTractsWithWallsVflow(N, l0, Sw, k, r)

    #Init
    rhoInit = model.rho0
    hInit = L0
    mInit = rhoInit*model.Sw*hInit

    init = np.zeros(model.n_state)

    for i in range(N):
        init[i*6+3] = Sw * model.rho0 * h0 # +Sw * model.rho0 * h0  * 2 * (i==0) 
        init[i*6+4] = h0# +h0 * 2 * (i==1)
    
    rpmsolver = solver.RPMSolverPHS(model, 1, 1, timestep, max_iter=10,  epsilon = 10**(-10), quad_order = 4)

    
    proj_input_signal = solver.adc_dac.discretize_Bo0(rpmsolver, input_signal, duration)

    xframes, dx_proj, w_proj, y_proj, lambda_proj, iters = rpmsolver.simulate(init, duration, proj_input_signal, f'results/dirac/40kHz/{N}_tracts_vflow.pkl')
    print(f'Simulation for N = {N} finished')

Mean number of NR iterations : 2.975
Max number of NR iterations : 3.0,              step index : 99
Simulation for N = 2 finished


## 17 tracts, fs = 20kHz, variable quadrature order

In [2]:
# (Some) geometry parameters and input signal
duration = 0.1
sr = 20000
timestep = 1/sr

# Dirac excitation
def input_signal(t):
    sig = np.zeros((len(t), model.n_io))
    sig[:, 0] =   2e-4 * (t<1/sr)
    return sig
    

N = 17
L0 = 0.01
k_s = 4e6
k = 845 #k_s*Sw
r = 0.8
h0 = 1e-2
l0 = 0.17/(2*N)
Sw = 2*l0*L0


In [3]:

# Model generation
model = models.MultipleTractsWithWallsVflow(N, l0, Sw, k, r)

# Initialization and input signals
rhoInit = model.rho0
hInit = L0
mInit = rhoInit*model.Sw*hInit

#Init
rhoInit = model.rho0
hInit = L0
mInit = rhoInit*model.Sw*hInit
init = np.zeros(model.n_state)
for i in range(N):
    init[i*6+3] = Sw * model.rho0 * h0 
    init[i*6+4] = h0 


In [5]:
quad_orders = [1] #2, 4, 8, 16, 32]
for quad_order in quad_orders:

    rpmsolver = solver.RPMSolverPHS(model, 1, 1, timestep, max_iter=200,  epsilon = 10**(-10), quad_order = quad_order)

    proj_input_signal = solver.adc_dac.discretize_Bo0(rpmsolver, input_signal, duration)

    xframes, dx_proj, w_proj, y_proj, lambda_proj, iters = rpmsolver.simulate(init, duration, proj_input_signal, f'results/dirac/quad_orders/{quad_order}.pkl')

Mean number of NR iterations : 3.0
Max number of NR iterations : 3.0,              step index : 0


## A nice simulation : 10 tracts, quad_order= 4, fs = 40000

In [5]:
# (Some) geometry parameters and input signal
duration = 0.5
sr = 40000
timestep = 1/sr

# Dirac excitation
def input_signal(t):
    sig = np.zeros((len(t), model.n_io))
    sig[:, 0] =   2e-4 * (t<1/sr)
    return sig
    

N = 10
L0 = 0.01
k_s = 4e6
k = 845 #k_s*Sw
r = 0.8
h0 = 1e-2
l0 = 0.17/(2*N)
Sw = 2*l0*L0

In [6]:
# Model generation
model = models.MultipleTractsWithWallsFull(N, l0, Sw, k, r)

# Initialization and input signals
rhoInit = model.rho0
hInit = L0
mInit = rhoInit*model.Sw*hInit

#Init
rhoInit = model.rho0
hInit = L0
mInit = rhoInit*model.Sw*hInit
init = np.zeros(model.n_state)
for i in range(N):
    init[i*6+3] = Sw * model.rho0 * h0 
    init[i*6+4] = h0 


In [7]:
rpmsolver = solver.RPMSolverPHS(model, 1, 1, timestep, max_iter=200,  epsilon = 10**(-10), quad_order = 4)

proj_input_signal = solver.adc_dac.discretize_Bo0(rpmsolver, input_signal, duration)

xframes, dx_proj, w_proj, y_proj, lambda_proj, iters = rpmsolver.simulate(init, duration, proj_input_signal, f'results/dirac/10_tracts_40k_quad4.pkl')

Mean number of NR iterations : 3.0
Max number of NR iterations : 3.0,              step index : 0


## Tests of vowel articulations

In [25]:
# (Some) geometry parameters and input signal
duration = 0.1
sr = 40000
timestep = 1/sr

# Noise excitation
def input_signal(t):
    sig = np.zeros((len(t), model.n_io))
    sig[:, 0] =   2e-4 * np.random.random(len(t))
    return sig
    

N = 10
L0 = 0.01
k_s = 4e6
k = 845 #k_s*Sw
r = 0.8
h0 = 1e-2
l0 = 0.17/(2*N)
Sw = 2*l0*L0

articulations = art.VowelArticulations(L0)
articulations.generate_profiles(N)

In [26]:
# Model generation
model = models.MultipleTractsWithWallsVflow(N, l0, Sw, k, r)

# Initialization and input signals
rhoInit = model.rho0
hInit = L0
mInit = rhoInit*model.Sw*hInit

vowel = "i"
#Init
rhoInit = model.rho0
hInit = L0
mInit = rhoInit*model.Sw*hInit
init = np.zeros(model.n_state)
for i in range(N):
    init[i*6+3] = Sw * model.rho0 * articulations.profile[vowel][i]
    init[i*6+4] = articulations.profile[vowel][i]


In [24]:
rpmsolver = solver.RPMSolverPHS(model, 1, 1, timestep, max_iter=200,  epsilon = 10**(-10), quad_order = 4)

proj_input_signal = solver.adc_dac.discretize_Bo0(rpmsolver, input_signal, duration)

xframes, dx_proj, w_proj, y_proj, lambda_proj, iters = rpmsolver.simulate(init, duration, proj_input_signal, f'results/vowels/{vowel}/10_40k_noise.pkl')

Interruption at t = 0.01815
